Run the notebook at TODAY after 15:00:00 (so the market is closed and data is available).

Input your current position.

The orders are for tomorrow.

In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
TODAY = str(cal.iloc[-1,0])
YESTERDAY = str(cal.iloc[-2,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-12-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-12-01
2024-12-11
2024-12-12


In [2]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 316
SH600570 800
SH603986 300
SZ002459 1300
SZ002129 2000
SZ000002 2300
SH600104 1100
SH600588 1500
SH600233 1000
SZ000999 300
'''



REC_LIST = [
    ('GBDT0', 'EXP_BENCH', 'c745fed5a5ec439f9abeafac76e750a5'), # GBDT BENCH_NOW r0 all
    # ('GBDT1', 'EXP_BENCH', 'ffe9470bec09417fb4db91128edc0a87'), # GBDT BENCH_NOW r1 all
    ('GBDT2', 'EXP_BENCH', 'e420ff30870d4e55a1c5e0fb207fa8a0'), # GBDT BENCH_A r0 all
    # ('GBDT3', 'EXP_BENCH', '1dcc7b1ec14e46379690c4f2d6e5242d'), # GBDT BENCH_A r1 all
    ('GBDT4', 'EXP_BENCH', '8d7e72af7f4b475f9f3646021a29bb1e'), # GBDT BENCH_B r0 all
    # ('GBDT5', 'EXP_BENCH', 'd308acba0eaf453b9f5acfc309b05c46'), # GBDT BENCH_B r1 all
    ('GBDT6', 'EXP_BENCH', 'ec4284016f6d44d897969410a5ef3ec0'), # GBDT BENCH_NOW r0 csi300
    # ('GBDT7', 'EXP_BENCH', 'c9dc8e870efc459396ea77850a7bc099'), # GBDT BENCH_NOW r1 csi300
    ('GBDT8', 'EXP_BENCH', '1f1711ba5caa4fd7a298196fd6a57558'), # GBDT BENCH_A r0 csi300
    # ('GBDT9', 'EXP_BENCH', 'a79446d3724942c58ed14c59622fbe2c'), # GBDT BENCH_A r1 csi300
    ('GBDT10', 'EXP_BENCH', '74d88afd5cb3448d831402b4e7571dc6'), # GBDT BENCH_B r0 csi300
    # ('GBDT5', 'EXP_BENCH', '58317fb6231c4ba0b4d8cecc07db822d'), # GBDT BENCH_B r1 csi300
]

# EXP_NAME, rid = REC_LIST[0]
TOPK = 5
NDROP = 1
HT = 0

info = {
    'ALGO': ['GBDT'],
    'market': ['csi300_ext'], 
    # 'market': ['all'], 
    'benchmark': ["SH000300"], 
    'feat': ["Alpha158"], 
    # 'feat': ["Alpha360"], 
    'label': ['r0'],
    'params': [f'topk {TOPK} ndrop {NDROP} HT {HT}']
}

In [3]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[3751819:MainThread](2024-12-13 09:39:43,664) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[3751819:MainThread](2024-12-13 09:39:43,668) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[3751819:MainThread](2024-12-13 09:39:43,669) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [4]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt)
print(account)
print(norm_pos)
print(hold_lst)

2024-12-12 09:00:00
{'SH600570': {'amount': 800}, 'SH603986': {'amount': 300}, 'SZ002459': {'amount': 1300}, 'SZ002129': {'amount': 2000}, 'SZ000002': {'amount': 2300}, 'SH600104': {'amount': 1100}, 'SH600588': {'amount': 1500}, 'SH600233': {'amount': 1000}, 'SZ000999': {'amount': 300}, 'cash': 316.0}
{'_settle_type': 'None', 'position': {'SH600588': {'amount': 7188, 'price': 2.604010820388794}, 'SH600104': {'amount': 12461, 'price': 1.537676215171814}, 'SZ000999': {'amount': 3640, 'price': 3.8244364261627197}, 'SH600570': {'amount': 3727, 'price': 6.821974754333496}, 'SZ000002': {'amount': 14867, 'price': 1.3443801403045654}, 'SZ002459': {'amount': 17480, 'price': 1.1809934377670288}, 'SH600233': {'amount': 7140, 'price': 2.0756261348724365}, 'SH603986': {'amount': 1783, 'price': 14.21588134765625}, 'SZ002129': {'amount': 4184, 'price': 4.798587799072266}, 'cash': 316.0, 'now_account_value': 178415.90483009815}, 'init_cash': 316.0}
['SH600588', 'SH600104', 'SZ000999', 'SH600570', 'SZ0

Prepare dataset

In [5]:
nameDFilter = NameDFilter(name_rule_re='(SH60[0-9]{4})|(SZ00[0-9]{4})')
filter_pipe=[nameDFilter]
benchmark = BENCH_Step(test_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

bt_benchmark = BENCH_Step(bt_split,
                        market=info['market'][0], \
                        benchmark=info['benchmark'][0], \
                        feat=info['feat'][0], \
                        label=info['label'][0], \
                        account=norm_account, \
                        filter_pipe=filter_pipe)

In [6]:
dataset = init_instance_by_config(benchmark.dataset_config)
bt_dataset = init_instance_by_config(bt_benchmark.dataset_config)
# dataset.prepare('test')

[3751819:MainThread](2024-12-13 09:40:31,967) INFO - qlib.timer - [log.py:127] - Time cost: 47.772s | Loading data Done
[3751819:MainThread](2024-12-13 09:40:31,974) INFO - qlib.timer - [log.py:127] - Time cost: 0.003s | DropnaLabel Done
[3751819:MainThread](2024-12-13 09:40:31,983) INFO - qlib.timer - [log.py:127] - Time cost: 0.007s | CSZScoreNorm Done
[3751819:MainThread](2024-12-13 09:40:31,984) INFO - qlib.timer - [log.py:127] - Time cost: 0.015s | fit & process data Done
[3751819:MainThread](2024-12-13 09:40:31,985) INFO - qlib.timer - [log.py:127] - Time cost: 47.791s | Init data Done
[3751819:MainThread](2024-12-13 09:41:20,098) INFO - qlib.timer - [log.py:127] - Time cost: 48.110s | Loading data Done
[3751819:MainThread](2024-12-13 09:41:20,109) INFO - qlib.timer - [log.py:127] - Time cost: 0.003s | DropnaLabel Done
[3751819:MainThread](2024-12-13 09:41:20,125) INFO - qlib.timer - [log.py:127] - Time cost: 0.015s | CSZScoreNorm Done
[3751819:MainThread](2024-12-13 09:41:20,126

In [7]:
label_df = bt_dataset.prepare("test")
label_df = pd.DataFrame(label_df.loc[:, 'LABEL0'])
label_df.columns = ['label']
# label_df.index = label_df.index.droplevel(0)
label_df

label
datetime   instrument          
2024-12-02 SH600000    0.016860
           SH600009   -0.000560
           SH600010   -0.005051
           SH600011    0.005666
           SH600015    0.030956
...                         ...
2024-12-11 SZ002841    0.039735
           SZ002916    0.004948
           SZ002920    0.009886
           SZ002938    0.007202
           SZ003816   -0.002463

[2056 rows x 1 columns]

Load and run model

In [64]:
def simp_eval(res_df, k, t=None, day=None):
    if day:
        date = str(cal.iloc[-day,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    # top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    top_k_per_group = sorted_df.groupby(['datetime']).apply(lambda x: x.nlargest(k, 'score'))#.reset_index(level=1, drop=1)
    if t:
        # top_k_per_group = top_k_per_group.tail(t)
        top_k_per_group = top_k_per_group.groupby(['datetime']).apply(lambda x: x.nsmallest(t, 'score'))
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    s, n = top_k_per_group['win'].sum(), len(top_k_per_group)
    # print(s, n)
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win

def run_model(model):
    BT_EXP_NAME = 'backtest'

    # backtest and analysis
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, bt_dataset, recorder)
        sr.generate()
    with R.start(experiment_name=BT_EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        bt_pred_df = recorder.load_object("pred.pkl")
    res_df = pd.concat([label_df, bt_pred_df], axis=1, sort=True).reindex( label_df.index)

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        # top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 10, day=day)
        # top25ic, top25win = simp_eval(res_df, 25, 5, day=day)
        top30ic, top30win = simp_eval(res_df, 30, 10, day=day)
        top40ic, top40win = simp_eval(res_df, 40, 10, day=day)
        top50ic, top50win = simp_eval(res_df, 50, 10, day=day)
        top60ic, top60win = simp_eval(res_df, 60, 10, day=day)
        top70ic, top70win = simp_eval(res_df, 70, 10, day=day)
        top80ic, top80win = simp_eval(res_df, 80, 10, day=day)        
        top90ic, top90win = simp_eval(res_df, 90, 10, day=day)
        top100ic, top100win = simp_eval(res_df, 100, 10, day=day)
        topic, topwin = simp_eval(res_df, 50, day=day)
        # return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win, top25ic, top25win, top30ic, top30win]
        return [topwin, top5win, top10win, top20win, top30win, top40win, top50win, top60win, top70win, top80win, top90win, top100win], [topic, top5ic, top10ic, top20ic, top30ic, top40ic, top50ic, top60ic, top70ic, top80ic, top90ic, top100ic]
    day1metrics = eval_day(2)
    day5metrics = eval_day(6)
    day10metrics = eval_day(11)
    day20metrics = eval_day(21)
    EXP_NAME = 'realworld_position_maker'

    # backtest and analysis
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):    
        recorder = R.get_recorder()
        ba_rid = R.get_recorder().id
        sr = SignalRecord(model, dataset, recorder)
        sr.generate()
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=ba_rid)
        pred_df = recorder.load_object("pred.pkl")

    return day1metrics, day5metrics, day10metrics, day20metrics, pred_df

# Run

In [65]:
info_win_dict1, info_ic_dict1 = {}, {}
info_win_dict5, info_ic_dict5 = {}, {}
info_win_dict10, info_ic_dict10 = {}, {}
info_win_dict20, info_ic_dict20 = {}, {}
pred_lst = []

for NAME, EXP_NAME, rid in REC_LIST:
    with R.start(experiment_name=EXP_NAME, uri='./mlrun'):
        recorder = R.get_recorder(recorder_id=rid)
        model = recorder.load_object("trained_model")
    day1metrics, day5metrics, day10metrics, day20metrics, pred_df = run_model(model)
    info_win_dict1[NAME], info_ic_dict1[NAME] = day1metrics
    info_win_dict5[NAME], info_ic_dict5[NAME] = day5metrics
    info_win_dict10[NAME], info_ic_dict10[NAME] = day10metrics
    info_win_dict20[NAME], info_ic_dict20[NAME] = day20metrics
    # for k in [1, 5, 10, 20]:
    #     eval(f'    info_win_dict{k}[NAME], info_ic_dict{k}[NAME] = day{k}metrics')
    pred_lst.append(pred_df)

[3751819:MainThread](2024-12-13 10:19:26,159) INFO - qlib.workflow - [exp.py:258] - Experiment 2 starts running ...
[3751819:MainThread](2024-12-13 10:19:26,171) INFO - qlib.workflow - [recorder.py:341] - Recorder 7ee442240f1d4b6fae2edc5b1b37f29d starts running under Experiment 2 ...
[3751819:MainThread](2024-12-13 10:19:26,344) INFO - qlib.timer - [log.py:127] - Time cost: 0.003s | waiting `async_log` Done
[3751819:MainThread](2024-12-13 10:19:26,350) INFO - qlib.workflow - [exp.py:258] - Experiment 4 starts running ...
[3751819:MainThread](2024-12-13 10:19:26,365) INFO - qlib.workflow - [recorder.py:341] - Recorder f0618cbe49dc4140adad68b60c4f761a starts running under Experiment 4 ...
[3751819:MainThread](2024-12-13 10:19:26,495) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 4
[3751819:MainThread](2024-12-13 10:19:26,503) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3751819:M

/home/linq/finance/private_trade/mlrun/1/c745fed5a5ec439f9abeafac76e750a5/artifacts/trained_model
trained_model
'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-02 SH600000   -0.007318
           SH600009   -0.005881
           SH600010    0.016440
           SH600011   -0.012365
           SH600015    0.002524


[3751819:MainThread](2024-12-13 10:19:26,525) INFO - qlib.workflow - [recorder.py:341] - Recorder ce5fc656c52540ae9b1ec1e9d1e2d70e starts running under Experiment 4 ...
[3751819:MainThread](2024-12-13 10:19:26,653) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done


/home/linq/finance/private_trade/mlrun/4/f0618cbe49dc4140adad68b60c4f761a/artifacts/pred.pkl
pred.pkl


[3751819:MainThread](2024-12-13 10:19:27,570) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3751819:MainThread](2024-12-13 10:19:27,591) INFO - qlib.workflow - [recorder.py:341] - Recorder 7e62c2720bf94ab2bf74c92e18b23835 starts running under Experiment 3 ...
[3751819:MainThread](2024-12-13 10:19:27,722) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 3
[3751819:MainThread](2024-12-13 10:19:27,732) INFO - qlib.timer - [log.py:127] - Time cost: 0.000s | waiting `async_log` Done
[3751819:MainThread](2024-12-13 10:19:27,740) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3751819:MainThread](2024-12-13 10:19:27,761) INFO - qlib.workflow - [recorder.py:341] - Recorder b5a0ea03004b4b03b44b755fd536deaf starts running under Experiment 3 ...
[3751819:MainThread](2024-12-13 10:19:27,887) INFO - qlib.timer - [log.py:127] - Time cost: 0.001s | waiting `async_log` Done
[3751819:M

...

/home/linq/finance/private_trade/mlrun/1/74d88afd5cb3448d831402b4e7571dc6/artifacts/trained_model
trained_model
'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-02 SH600000   -0.038353
           SH600009    0.004608
           SH600010    0.032711
           SH600011    0.007540
           SH600015    0.040062


[3751819:MainThread](2024-12-13 10:19:35,358) INFO - qlib.workflow - [recorder.py:341] - Recorder 4bbf6201e46e4b88979d661e9af9215b starts running under Experiment 4 ...
[3751819:MainThread](2024-12-13 10:19:35,478) INFO - qlib.timer - [log.py:127] - Time cost: 0.001s | waiting `async_log` Done


/home/linq/finance/private_trade/mlrun/4/f9b499dd3dc04b7899c07d84e4e6ac17/artifacts/pred.pkl
pred.pkl


[3751819:MainThread](2024-12-13 10:19:36,395) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3751819:MainThread](2024-12-13 10:19:36,419) INFO - qlib.workflow - [recorder.py:341] - Recorder f7bac6d5415e4019b320b884170af83d starts running under Experiment 3 ...
[3751819:MainThread](2024-12-13 10:19:36,556) INFO - qlib.workflow - [record_temp.py:198] - Signal record 'pred.pkl' has been saved as the artifact of the Experiment 3
[3751819:MainThread](2024-12-13 10:19:36,565) INFO - qlib.timer - [log.py:127] - Time cost: 0.002s | waiting `async_log` Done
[3751819:MainThread](2024-12-13 10:19:36,573) INFO - qlib.workflow - [exp.py:258] - Experiment 3 starts running ...
[3751819:MainThread](2024-12-13 10:19:36,595) INFO - qlib.workflow - [recorder.py:341] - Recorder 5adb264e6af541bca1c7fb59846275db starts running under Experiment 3 ...
[3751819:MainThread](2024-12-13 10:19:36,719) INFO - qlib.timer - [log.py:127] - Time cost: 0.003s | waiting `async_log` Done


'The following are prediction results of the LGBModel model.'
                          score
datetime   instrument          
2024-12-12 SH600000    0.030818
           SH600009    0.029509
           SH600010   -0.031014
           SH600011    0.009100
           SH600015   -0.049552
/home/linq/finance/private_trade/mlrun/3/f7bac6d5415e4019b320b884170af83d/artifacts/pred.pkl
pred.pkl


In [66]:
pred_df = pd.concat(pred_lst, axis=1)
pred_df.columns = name_lst = [r[0] for r in REC_LIST]
_pred_df = pred_df

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([pred_df[n]*info_win_dict5[n][0] for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [67]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df

In [68]:
hold_lst

['SH600588',
 'SH600104',
 'SZ000999',
 'SH600570',
 'SZ000002',
 'SZ002459',
 'SH600233',
 'SH603986',
 'SZ002129']

In [69]:
pos_rank = rank_df.sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
report_pos = pos_rank

In [70]:
color_lst = [n for n in report_pos.columns if n not in ['close']]
# color_lst

In [71]:
sty_report_pos = report_pos.style.background_gradient(cmap='Blues_r', subset=color_lst)
sty_report_pos

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SH600104,-0.045602,-0.095787,-0.118792,0.016905,-0.042704,-0.030952,-0.052822,0.044145,-0.202378,17.420000
SZ000999,-0.030453,-0.044142,-0.068137,-0.039849,-0.040063,-0.056775,-0.046570,0.012399,-0.182784,46.410000
SH600233,-0.000358,-0.012536,-0.008486,-0.019066,-0.030283,-0.007431,-0.013027,0.009547,-0.051617,14.820000
SH600588,0.007686,0.026450,0.067287,-0.008885,0.000845,0.090871,0.030709,0.036435,0.121203,12.480000
SZ002129,0.038912,0.059106,0.103770,0.044926,0.041208,0.085768,0.062282,0.024408,0.244332,10.040000
SH600570,0.044868,0.140914,0.094702,0.011965,0.043290,0.109030,0.074128,0.044297,0.287809,31.790001
SZ000002,0.060523,0.080939,0.119514,0.025535,0.049117,0.128262,0.077315,0.036851,0.302624,8.690001
SZ002459,0.025286,0.062513,0.120249,0.053443,0.071341,0.138618,0.078575,0.038995,0.310503,15.880000
SH603986,0.060442,0.088996,0.138719,0.049820,0.069724,0.205405,0.102184,0.054316,0.402900,84.529999


In [72]:
sty_report_top0 = report_top.head(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top1 = report_top.head(20).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top2 = report_top.head(30).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top3 = report_top.head(40).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top4 = report_top.head(50).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top5 = report_top.head(60).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top6 = report_top.head(70).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top7 = report_top.head(80).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top8 = report_top.head(90).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top9 = report_top.head(100).tail(10).style.background_gradient(axis=0, subset=color_lst)

In [73]:
sty_report_top0

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SH600438,0.061138,0.157182,0.174471,0.126100,0.145689,0.228155,0.148789,0.050392,0.586506,25.250000
SH601100,0.081204,0.132864,0.087524,0.087664,0.139130,0.337093,0.144246,0.089199,0.571978,56.380001
SH600732,0.051740,0.029150,0.121064,0.120881,0.121996,0.248126,0.115493,0.069778,0.461246,13.610000
SH600745,0.069730,0.168333,0.188234,0.045618,0.049114,0.162572,0.113934,0.060095,0.443932,36.599998
SH600048,0.061708,0.095072,0.139104,0.029058,0.067327,0.237757,0.105004,0.068248,0.414124,10.210000
SH603986,0.060442,0.088996,0.138719,0.049820,0.069724,0.205405,0.102184,0.054316,0.402900,84.529999
SH601865,0.043070,0.089368,0.143142,0.065557,0.123044,0.122984,0.097861,0.035198,0.384868,23.959999
SZ001965,0.058660,0.112003,0.083774,0.052086,0.092726,0.132605,0.088642,0.028149,0.347686,12.370000
SZ002179,0.035568,0.096684,0.119591,0.061931,0.096402,0.100636,0.085135,0.027938,0.334114,40.169998


In [74]:
sty_report_top1

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SH601155,0.051433,0.127940,0.134443,0.005289,0.017493,0.141811,0.079735,0.056844,0.310209,13.790001
SZ000002,0.060523,0.080939,0.119514,0.025535,0.049117,0.128262,0.077315,0.036851,0.302624,8.690001
SZ000733,0.017740,0.053585,0.059921,0.024914,0.029421,0.257506,0.073848,0.083518,0.295493,46.190002
SH601211,0.045629,0.107636,0.085575,0.024967,0.045614,0.135129,0.074092,0.038766,0.290011,19.430000
SH600570,0.044868,0.140914,0.094702,0.011965,0.043290,0.109030,0.074128,0.044297,0.287809,31.790001
SZ002736,0.047778,0.043372,0.087557,0.025762,0.049461,0.169941,0.070645,0.048090,0.279691,11.630000
SH601009,0.050505,0.069478,0.085528,0.057602,0.080847,0.080142,0.070683,0.012864,0.277340,10.620000
SH600893,0.054288,0.067636,0.095647,0.067401,0.070615,0.062108,0.069616,0.012772,0.272610,41.230000
SZ002027,0.060134,0.101175,0.127116,0.010688,0.040368,0.069596,0.068179,0.038096,0.263791,7.040000


In [75]:
sty_report_top2

,GBDT0,GBDT2,GBDT4,GBDT6,GBDT8,GBDT10,mean,std,score,close
instrument,,,,,,,,,,
SZ000938,0.038761,0.072957,0.092148,0.012755,0.031454,0.130595,0.063112,0.040052,0.247649,25.260002
SZ002129,0.038912,0.059106,0.103770,0.044926,0.041208,0.085768,0.062282,0.024408,0.244332,10.040000
SH601236,0.029615,0.063248,0.070496,0.032012,0.041556,0.122569,0.059916,0.031839,0.236291,8.760000
SH601838,0.048502,0.087099,0.068434,0.040994,0.046411,0.070006,0.060241,0.016245,0.234915,16.219999
SH600900,0.039239,0.055335,0.065104,0.056276,0.079843,0.050109,0.057651,0.012591,0.226277,28.400000
SH600837,0.040362,0.087899,0.061478,0.015303,0.025352,0.116426,0.057803,0.035401,0.226150,11.780000
SH601021,0.017245,0.049051,0.062752,0.017273,0.022446,0.168903,0.056278,0.053179,0.223764,58.600002
SH600989,0.018195,0.035908,0.015039,0.050149,0.062408,0.149372,0.055179,0.045277,0.221013,16.269999
SZ002241,0.100141,0.033390,0.008940,0.082925,0.079915,0.026754,0.055344,0.033724,0.216699,27.400002


In [76]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

dfi.export(infodf, './tmp/ensemble_info.png',table_conversion='matplotlib')

# dfi.export(sty_report_info5, './tmp/ensemble_performance5.png',table_conversion='matplotlib')
# dfi.export(sty_report_info10, './tmp/ensemble_performance10.png',table_conversion='matplotlib')
# dfi.export(sty_report_info20, './tmp/ensemble_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_top0.png',table_conversion='matplotlib')
dfi.export(sty_report_top1, './tmp/ensemble_top1.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_top2.png',table_conversion='matplotlib')
dfi.export(sty_report_top3, './tmp/ensemble_top3.png',table_conversion='matplotlib')
dfi.export(sty_report_top4, './tmp/ensemble_top4.png',table_conversion='matplotlib')
dfi.export(sty_report_top5, './tmp/ensemble_top5.png',table_conversion='matplotlib')
dfi.export(sty_report_top6, './tmp/ensemble_top6.png',table_conversion='matplotlib')
dfi.export(sty_report_top7, './tmp/ensemble_top7.png',table_conversion='matplotlib')
dfi.export(sty_report_top8, './tmp/ensemble_top8.png',table_conversion='matplotlib')
dfi.export(sty_report_top9, './tmp/ensemble_top9.png',table_conversion='matplotlib')
dfi.export(sty_report_pos, './tmp/ensemble_watch.png',table_conversion='matplotlib')

In [77]:

for k in [1,5,10,20]:
    sty_report_win_info = eval(f"pd.DataFrame(info_win_dict{k}, index=['TOP50WIN', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100'])").style.background_gradient(axis=0)
    sty_report_ic_info = eval(f"pd.DataFrame(info_ic_dict{k}, index=['TOP50IC', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100'])").style.background_gradient(axis=0)
    dfi.export(sty_report_win_info, f'./tmp/ensemble_win{k}.png',table_conversion='matplotlib')
    dfi.export(sty_report_ic_info, f'./tmp/ensemble_ic{k}.png',table_conversion='matplotlib')